In [1]:
import os
import numpy as np
import torch
import deepirtools
from deepirtools import IWAVE
from factor_analyzer import Rotator
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
plt.rcParams['figure.figsize'] = (15, 12)
deepirtools.manual_seed(123) # Seed for reproducibility.

# Comparing by gender

## Men

In [2]:
group = 1           # age class 'AK'
test_size = 0.2     # train-test split
sub = False         # subjective meassures
save_dir = './paper_models_test'

dat = pd.read_csv('../data/merged_male.csv')

y_0 = dat['LZ']
dat = dat.drop(['TalentID', 'AK', 'LZ'], axis=1).reset_index(drop=True)
X_0 = dat.iloc[:,:]

# # english variable-names
X_names = ['relative age', 'height', 'weight', 'sprint (20m)', 'agility','dribbling','ball control','juggling','tactical skills','kicking skills','endurance','psychological skills']
# X_0 = dat.iloc[:, 2:c]
# y_0 = dat.iloc[:, 16]
X_0.columns = X_names


#X, X_test, y, y_test = train_test_split(X_0, y_0, test_size=test_size, random_state=42, stratify=y_0)
#X.columns = X_names


# ___ SCALING
# MinMax [0,1]
#scaler_train = StandardScaler().fit(X)
#df_train = scaler_train.transform(X)
#y_train = np.asarray(y)
#df_test = scaler_train.transform(X_test)

# torch
#dft = torch.tensor(df_train)
#dftest = torch.tensor(df_test)
#sample_size = df_train.shape[0]
#test_size = df_test.shape[0]
#Y = dft.view(sample_size, -1)
#Ytest = dftest.view(test_size, -1)

# Full data
X_full = X_0.copy()
y_full = y_0.copy()

scaler = StandardScaler().fit(X_full)
df = scaler.transform(X_full)
y = np.asarray(y_full)

dft = torch.tensor(df)
sample_size = df.shape[0]
Y = dft.view(sample_size, -1)
n_items = Y.shape[1]

In [ ]:
latent_size = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#ref_idxs = Y.var(dim = 0).sort(descending = True)[1][:latent_size] # Find anchor items.
ref_idxs = [1,3,6,8]
A = torch.ones(latent_size * n_items)
b = torch.zeros(latent_size * n_items)
for factor_idx in range(latent_size):
    ref_idx = ref_idxs[factor_idx]
    A[n_items * factor_idx + ref_idx] = 0
    b[n_items * factor_idx + ref_idx] = 1 # Fix anchor item's loading on one factor to one.
A = A.diag()
ints_mask = torch.zeros(n_items)

model_name = 'iwave_male'
inf_net = [15,8]

# model
iwave = IWAVE(model_type = "normal",
              learning_rate = 5e-4 if device=="cpu" else 1e-3,
              gradient_estimator='dreg',
              n_intervals = 100,
              device = device,
              A = A,
              b = b,
              ints_mask = ints_mask,
              inference_net_sizes = inf_net,
              latent_size = latent_size,
              n_items = n_items,
              fixed_variances = False,
              fixed_means = False,
              use_spline_prior = False,
              log_interval = 25,
              )

iwave.load_model(model_name, "./saved_models")

In [ ]:
# nIWAVE
model_name = 'niwave_u13'
load_model = os.path.exists(save_dir + '/' + model_name + '.pth')

# specs
latent_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ref_idxs = [1,3,6,8]
A = torch.ones(latent_size * n_items)
b = torch.zeros(latent_size * n_items)
for factor_idx in range(latent_size):
    ref_idx = ref_idxs[factor_idx]
    A[n_items * factor_idx + ref_idx] = 0
    b[n_items * factor_idx + ref_idx] = 1 # Fix anchor item's loading on one factor to one.
A = A.diag()
ints_mask = torch.zeros(n_items)
inf_net = [15,8]

#### Load second model
model_name = 'niwave_male'
inf_net = [15,8]

niwave = IWAVE(model_type = "normal",
              learning_rate = 5e-4 if device=="cpu" else 1e-3, # Bigger learning rate with increased
              gradient_estimator='dreg',
              n_intervals = 100,
              device = device,
              A = A,
              b = b,
              ints_mask = ints_mask,
              inference_net_sizes = inf_net,
              latent_size = latent_size,
              n_items = n_items,
              fixed_variances = False,
              fixed_means = False,
              use_spline_prior = True,
              count_bins = 32,
              flow_length = 2,
              log_interval = 25,
              )

niwave.load_model(model_name, "./saved_models")

#### Log-likelihood

In [ ]:
#Xt_test = torch.tensor(df_test)
#test_size = Xt_test.shape[0]
#Y_test = Xt_test.view(test_size, -1)

ll = iwave.log_likelihood(data = Y, iw_samples = 5000)
nll = niwave.log_likelihood(data = Y, iw_samples = 5000) # Increasing iw_samples
print(-ll/sample_size,-nll/sample_size)

#### Scores

In [ ]:
scores_iwave = iwave.scores(Y, mc_samples=50, iw_samples=50)
scores_iwave

In [ ]:
scores_niwave = niwave.scores(Y, mc_samples=50, iw_samples=50)
scores_niwave

In [ ]:
# why this order of index?
final = pd.DataFrame(dat)
final['dim1_iwave'] = scores_iwave[:,3]
final['dim2_iwave'] = scores_iwave[:,0]
final['dim3_iwave'] = scores_iwave[:,2]
final['dim4_iwave'] = scores_iwave[:,1]
final['dim1_niwave'] = scores_niwave[:,3]
final['dim2_niwave'] = scores_niwave[:,0]
final['dim3_niwave'] = scores_niwave[:,1]
final['dim4_niwave'] = scores_niwave[:,2]
final['y'] = y

scaled_names = ['relative age scaled',
                'height scaled',
                'weight scaled',
                'sprint (20m) scaled',
                'agility scaled',
                'dribbling scaled',
                'ball control scaled',
                'juggling scaled',
                'tactical skills scaled',
                'kicking skills scaled',
                'endurance scaled',
                'psychological skills scaled']

final = pd.concat([final,pd.DataFrame(df, columns=scaled_names)],axis=1)
final

In [ ]:
final.to_csv(save_dir+'/data_scores', index=False)

In [ ]:
succ = final[final['y']==1]
fail = final[final['y']==0]
succ_scores = succ.iloc[:,11:15]
fail_scores = fail.iloc[:,11:15]

succ_scores

In [ ]:
from scipy.stats import ttest_ind
test_result = np.transpose(succ.iloc[:, 1:2].describe().iloc[[1,2], :])
temp_0 = np.transpose(fail.iloc[:, 1:2].describe().iloc[[1,2], :])
stat, p = ttest_ind(succ.iloc[:, 1],fail.iloc[:, 1])
test_result[['mean_0', 'std_0']] = temp_0
test_result["t-statistic"] = stat
test_result["p-value"] = p
for i in range(2, final.iloc[:, 1::].shape[1]):
    stat, p = ttest_ind(succ[final.columns[i]],fail[final.columns[i]])
    temp = np.transpose(succ.iloc[:, i:(i+1)].describe().iloc[[1,2], :])
    temp_0 = np.transpose(fail.iloc[:, i:(i+1)].describe().iloc[[1,2], :])
    temp[['mean_0', 'std_0']] = temp_0
    temp["t-statistic"] = stat
    temp["p-value"] = p
    test_result = pd.concat([test_result,temp],axis=0)
    # test_result = test_result.append(temp)
    # print('For ', name, 'the test statistic is %.3f and p=%.3f' % (stat, p))
test_result

### PCA Comparision

In [ ]:
from sklearn.decomposition import PCA
#pca_scree = PCA(n_components = 0.75, svd_solver='full')
pca_scree = PCA(svd_solver='full')
pca_scree.fit(df)
reduced = pca_scree.transform(df)
pca_scree.explained_variance_

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.plot(pca_scree.explained_variance_ratio_, marker='o')
plt.xlabel("Eigenvalue number")
plt.ylabel("Eigenvalue size")
plt.title("Scree Plot")

In [ ]:
pca = PCA(n_components = 4, svd_solver='full',)
pca.fit(df)
reduced = pca.transform(df)
loadings = pd.DataFrame(pca.components_.T)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(25,12))
sns.heatmap(ax=axs[0], data=loadings, annot=True, linewidths=0.5, vmin=loadings.min().min(), vmax=loadings.max().max(), center=0,xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names).set(title='PCA loadings')
sns.heatmap(ax=axs[1], data=rot_loadings_iwave[:,[3,0,2,1]], annot=True, linewidths=0.5, vmin=rot_loadings_iwave.min(), vmax=rot_loadings_iwave.max(), center=0, xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names).set(title='Loadings oblique')
sns.heatmap(ax=axs[2], data=rot_loadings_niwave[:,[3,0,1,2,]], annot=True, linewidths=0.5, vmin=rot_loadings_niwave.min(), vmax=rot_loadings_niwave.max(), center=0, xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names).set(title='Loadings oblique_spl')

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(16,7), frameon=True)
sns.heatmap(ax=axs[0], data=abs(loadings), annot=False, linewidths=0.5, linecolor='black', vmin=0, center=loadings.max().max()/2, vmax=loadings.max().max(),xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names, cmap = plt.cm.Greys, cbar=True, cbar_kws={'format': matplotlib.ticker.FixedFormatter(['0','max']),"ticks":[0,loadings.max().max()],}).set(title='PCA')
sns.heatmap(ax=axs[1], data=abs(rot_loadings_iwave[:,[3,0,2,1]]), annot=False, linewidths=0.5, linecolor='black', vmin=0, center=rot_loadings_iwave.max()/2, vmax=rot_loadings_iwave.max(), xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=False, cmap = plt.cm.Greys, cbar=True, cbar_kws={'format': matplotlib.ticker.FixedFormatter(['0','max']),"ticks":[0,rot_loadings_iwave.max()],}).set(title='Linear loadings')
sns.heatmap(ax=axs[2], data=abs(rot_loadings_niwave[:,[3,0,1,2,]]), annot=False, linewidths=0.5, linecolor='black', vmin=0, center=(rot_loadings_niwave.max()-0.7)/2,vmax=rot_loadings_niwave.max()-0.7, xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=False, cmap = plt.cm.Greys, cbar_kws={'format': matplotlib.ticker.FixedFormatter(['0','max']),"ticks":[0,rot_loadings_niwave.max()-0.7],}).set(title='Nonlinear loadings')

In [ ]:
rot_load_table = torch.clone(rot_loadings_iwave)
rot_load_table_spl = torch.clone(rot_loadings_niwave)

loadings_table = loadings.copy()
loadings_table['variable'] = X_names
loadings_table.set_index('variable', inplace=True)
loadings_table.columns = ['fact. 1', 'fact. 2', 'fact. 3', 'fact. 4']

loadings_table[['iwave fact. 1', 'iwave fact. 2', 'iwave fact. 3', 'iwave fact. 4']] = rot_loadings_iwave[:,[3,0,2,1]]
loadings_table[['niwave fact. 1', 'niwave fact. 2', 'niwave fact. 3', 'niwave fact. 4']] = rot_loadings_niwave[:,[3,0,1,2,]]

loadings_table = round(loadings_table,2)
cut_percent = 0.1
pca_cut = cut_percent*loadings.max().max()
iwave_cut = np.asarray(cut_percent*rot_loadings_iwave.max())
niwave_cut = np.asarray(cut_percent*(rot_loadings_niwave.max()-0.7))
# pca
loadings_table.loc[np.abs(loadings_table['fact. 1']) < pca_cut, 'fact. 1'] = '.'
loadings_table.loc[np.abs(loadings_table['fact. 2']) < pca_cut, 'fact. 2'] = '.'
loadings_table.loc[np.abs(loadings_table['fact. 3']) < pca_cut, 'fact. 3'] = '.'
loadings_table.loc[np.abs(loadings_table['fact. 4']) < pca_cut, 'fact. 4'] = '.'
# iwave
loadings_table.loc[np.abs(loadings_table['iwave fact. 1']) < iwave_cut, 'iwave fact. 1'] = '.'
loadings_table.loc[np.abs(loadings_table['iwave fact. 2']) < iwave_cut, 'iwave fact. 2'] = '.'
loadings_table.loc[np.abs(loadings_table['iwave fact. 3']) < iwave_cut, 'iwave fact. 3'] = '.'
loadings_table.loc[np.abs(loadings_table['iwave fact. 4']) < iwave_cut, 'iwave fact. 4'] = '.'
# niwave
loadings_table.loc[np.abs(loadings_table['niwave fact. 1']) < niwave_cut, 'niwave fact. 1'] = '.'
loadings_table.loc[np.abs(loadings_table['niwave fact. 2']) < niwave_cut, 'niwave fact. 2'] = '.'
loadings_table.loc[np.abs(loadings_table['niwave fact. 3']) < niwave_cut, 'niwave fact. 3'] = '.'
loadings_table.loc[np.abs(loadings_table['niwave fact. 4']) < niwave_cut, 'niwave fact. 4'] = '.'


print(round(loadings_table,2).to_latex(escape=True, index=True, index_names=False, column_format='lrrrrrrrrrrrr', ))

## Women

In [ ]:
group = 1           # age class 'AK'
test_size = 0.2     # train-test split
sub = False         # subjective meassures
save_dir = './paper_models_test'

dat = pd.read_csv('../data/merged_female.csv')

y_0 = dat['LZ']
dat = dat.drop(['TalentID', 'AK', 'LZ'], axis=1).reset_index(drop=True)
X_0 = dat.iloc[:,:]

# # english variable-names
X_names = ['relative age', 'height', 'weight', 'sprint (20m)', 'agility','dribbling','ball control','juggling','tactical skills','kicking skills','endurance','psychological skills']
# X_0 = dat.iloc[:, 2:c]
# y_0 = dat.iloc[:, 16]
X_0.columns = X_names


#X, X_test, y, y_test = train_test_split(X_0, y_0, test_size=test_size, random_state=42, stratify=y_0)
#X.columns = X_names


# ___ SCALING
# MinMax [0,1]
#scaler_train = StandardScaler().fit(X)
#df_train = scaler_train.transform(X)
#y_train = np.asarray(y)
#df_test = scaler_train.transform(X_test)

# torch
#dft = torch.tensor(df_train)
#dftest = torch.tensor(df_test)
#sample_size = df_train.shape[0]
#test_size = df_test.shape[0]
#Y = dft.view(sample_size, -1)
#Ytest = dftest.view(test_size, -1)

# Full data
X_full = X_0.copy()
y_full = y_0.copy()

scaler = StandardScaler().fit(X_full)
df = scaler.transform(X_full)
y = np.asarray(y_full)

dft = torch.tensor(df)
sample_size = df.shape[0]
Y = dft.view(sample_size, -1)
n_items = Y.shape[1]

In [ ]:
latent_size = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#ref_idxs = Y.var(dim = 0).sort(descending = True)[1][:latent_size] # Find anchor items.
ref_idxs = [1,3,6,8]
A = torch.ones(latent_size * n_items)
b = torch.zeros(latent_size * n_items)
for factor_idx in range(latent_size):
    ref_idx = ref_idxs[factor_idx]
    A[n_items * factor_idx + ref_idx] = 0
    b[n_items * factor_idx + ref_idx] = 1 # Fix anchor item's loading on one factor to one.
A = A.diag()
ints_mask = torch.zeros(n_items)

model_name = 'iwave_female'
inf_net = [15,8]

# model
iwave = IWAVE(model_type = "normal",
              learning_rate = 5e-4 if device=="cpu" else 1e-3,
              gradient_estimator='dreg',
              n_intervals = 100,
              device = device,
              A = A,
              b = b,
              ints_mask = ints_mask,
              inference_net_sizes = inf_net,
              latent_size = latent_size,
              n_items = n_items,
              fixed_variances = False,
              fixed_means = False,
              use_spline_prior = False,
              log_interval = 25,
              )

iwave.load_model(model_name, "./saved_models")

In [ ]:
# nIWAVE
model_name = 'niwave_u13'
load_model = os.path.exists(save_dir + '/' + model_name + '.pth')

# specs
latent_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ref_idxs = [1,3,6,8]
A = torch.ones(latent_size * n_items)
b = torch.zeros(latent_size * n_items)
for factor_idx in range(latent_size):
    ref_idx = ref_idxs[factor_idx]
    A[n_items * factor_idx + ref_idx] = 0
    b[n_items * factor_idx + ref_idx] = 1 # Fix anchor item's loading on one factor to one.
A = A.diag()
ints_mask = torch.zeros(n_items)
inf_net = [15,8]

#### Load second model
model_name = 'niwave_female'
inf_net = [15,8]

niwave = IWAVE(model_type = "normal",
              learning_rate = 5e-4 if device=="cpu" else 1e-3, # Bigger learning rate with increased
              gradient_estimator='dreg',
              n_intervals = 100,
              device = device,
              A = A,
              b = b,
              ints_mask = ints_mask,
              inference_net_sizes = inf_net,
              latent_size = latent_size,
              n_items = n_items,
              fixed_variances = False,
              fixed_means = False,
              use_spline_prior = True,
              count_bins = 32,
              flow_length = 2,
              log_interval = 25,
              )

niwave.load_model(model_name, "./saved_models")

#### Log-likelihood

In [ ]:
#Xt_test = torch.tensor(df_test)
#test_size = Xt_test.shape[0]
#Y_test = Xt_test.view(test_size, -1)

ll = iwave.log_likelihood(data = Y, iw_samples = 5000)
nll = niwave.log_likelihood(data = Y, iw_samples = 5000) # Increasing iw_samples
print(-ll/sample_size,-nll/sample_size)

#### Scores

In [ ]:
scores_iwave = iwave.scores(Y, mc_samples=50, iw_samples=50)
scores_iwave

In [ ]:
scores_niwave = niwave.scores(Y, mc_samples=50, iw_samples=50)
scores_niwave

In [ ]:
# why this order of index?
final = pd.DataFrame(dat)
final['dim1_iwave'] = scores_iwave[:,3]
final['dim2_iwave'] = scores_iwave[:,0]
final['dim3_iwave'] = scores_iwave[:,2]
final['dim4_iwave'] = scores_iwave[:,1]
final['dim1_niwave'] = scores_niwave[:,3]
final['dim2_niwave'] = scores_niwave[:,0]
final['dim3_niwave'] = scores_niwave[:,1]
final['dim4_niwave'] = scores_niwave[:,2]
final['y'] = y

scaled_names = ['relative age scaled',
                'height scaled',
                'weight scaled',
                'sprint (20m) scaled',
                'agility scaled',
                'dribbling scaled',
                'ball control scaled',
                'juggling scaled',
                'tactical skills scaled',
                'kicking skills scaled',
                'endurance scaled',
                'psychological skills scaled']

final = pd.concat([final,pd.DataFrame(df, columns=scaled_names)],axis=1)
final

In [ ]:
final.to_csv(save_dir+'/data_scores', index=False)

In [ ]:
succ = final[final['y']==1]
fail = final[final['y']==0]
succ_scores = succ.iloc[:,11:15]
fail_scores = fail.iloc[:,11:15]

succ_scores

In [ ]:
from scipy.stats import ttest_ind
test_result = np.transpose(succ.iloc[:, 1:2].describe().iloc[[1,2], :])
temp_0 = np.transpose(fail.iloc[:, 1:2].describe().iloc[[1,2], :])
stat, p = ttest_ind(succ.iloc[:, 1],fail.iloc[:, 1])
test_result[['mean_0', 'std_0']] = temp_0
test_result["t-statistic"] = stat
test_result["p-value"] = p
for i in range(2, final.iloc[:, 1::].shape[1]):
    stat, p = ttest_ind(succ[final.columns[i]],fail[final.columns[i]])
    temp = np.transpose(succ.iloc[:, i:(i+1)].describe().iloc[[1,2], :])
    temp_0 = np.transpose(fail.iloc[:, i:(i+1)].describe().iloc[[1,2], :])
    temp[['mean_0', 'std_0']] = temp_0
    temp["t-statistic"] = stat
    temp["p-value"] = p
    test_result = pd.concat([test_result,temp],axis=0)
    # test_result = test_result.append(temp)
    # print('For ', name, 'the test statistic is %.3f and p=%.3f' % (stat, p))
test_result

### PCA Comparision

In [ ]:
from sklearn.decomposition import PCA
#pca_scree = PCA(n_components = 0.75, svd_solver='full')
pca_scree = PCA(svd_solver='full')
pca_scree.fit(df)
reduced = pca_scree.transform(df)
pca_scree.explained_variance_

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.plot(pca_scree.explained_variance_ratio_, marker='o')
plt.xlabel("Eigenvalue number")
plt.ylabel("Eigenvalue size")
plt.title("Scree Plot")

In [ ]:
pca = PCA(n_components = 4, svd_solver='full',)
pca.fit(df)
reduced = pca.transform(df)
loadings = pd.DataFrame(pca.components_.T)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(25,12))
sns.heatmap(ax=axs[0], data=loadings, annot=True, linewidths=0.5, vmin=loadings.min().min(), vmax=loadings.max().max(), center=0,xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names).set(title='PCA loadings')
sns.heatmap(ax=axs[1], data=rot_loadings_iwave[:,[3,0,2,1]], annot=True, linewidths=0.5, vmin=rot_loadings_iwave.min(), vmax=rot_loadings_iwave.max(), center=0, xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names).set(title='Loadings oblique')
sns.heatmap(ax=axs[2], data=rot_loadings_niwave[:,[3,0,1,2,]], annot=True, linewidths=0.5, vmin=rot_loadings_niwave.min(), vmax=rot_loadings_niwave.max(), center=0, xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names).set(title='Loadings oblique_spl')

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(16,7), frameon=True)
sns.heatmap(ax=axs[0], data=abs(loadings), annot=False, linewidths=0.5, linecolor='black', vmin=0, center=loadings.max().max()/2, vmax=loadings.max().max(),xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=X_names, cmap = plt.cm.Greys, cbar=True, cbar_kws={'format': matplotlib.ticker.FixedFormatter(['0','max']),"ticks":[0,loadings.max().max()],}).set(title='PCA')
sns.heatmap(ax=axs[1], data=abs(rot_loadings_iwave[:,[3,0,2,1]]), annot=False, linewidths=0.5, linecolor='black', vmin=0, center=rot_loadings_iwave.max()/2, vmax=rot_loadings_iwave.max(), xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=False, cmap = plt.cm.Greys, cbar=True, cbar_kws={'format': matplotlib.ticker.FixedFormatter(['0','max']),"ticks":[0,rot_loadings_iwave.max()],}).set(title='Linear loadings')
sns.heatmap(ax=axs[2], data=abs(rot_loadings_niwave[:,[3,0,1,2,]]), annot=False, linewidths=0.5, linecolor='black', vmin=0, center=(rot_loadings_niwave.max()-0.7)/2,vmax=rot_loadings_niwave.max()-0.7, xticklabels=["Factor 1", "Factor 2","Factor 3","Factor 4"], yticklabels=False, cmap = plt.cm.Greys, cbar_kws={'format': matplotlib.ticker.FixedFormatter(['0','max']),"ticks":[0,rot_loadings_niwave.max()-0.7],}).set(title='Nonlinear loadings')

In [ ]:
rot_load_table = torch.clone(rot_loadings_iwave)
rot_load_table_spl = torch.clone(rot_loadings_niwave)

loadings_table = loadings.copy()
loadings_table['variable'] = X_names
loadings_table.set_index('variable', inplace=True)
loadings_table.columns = ['fact. 1', 'fact. 2', 'fact. 3', 'fact. 4']

loadings_table[['iwave fact. 1', 'iwave fact. 2', 'iwave fact. 3', 'iwave fact. 4']] = rot_loadings_iwave[:,[3,0,2,1]]
loadings_table[['niwave fact. 1', 'niwave fact. 2', 'niwave fact. 3', 'niwave fact. 4']] = rot_loadings_niwave[:,[3,0,1,2,]]

loadings_table = round(loadings_table,2)
cut_percent = 0.1
pca_cut = cut_percent*loadings.max().max()
iwave_cut = np.asarray(cut_percent*rot_loadings_iwave.max())
niwave_cut = np.asarray(cut_percent*(rot_loadings_niwave.max()-0.7))
# pca
loadings_table.loc[np.abs(loadings_table['fact. 1']) < pca_cut, 'fact. 1'] = '.'
loadings_table.loc[np.abs(loadings_table['fact. 2']) < pca_cut, 'fact. 2'] = '.'
loadings_table.loc[np.abs(loadings_table['fact. 3']) < pca_cut, 'fact. 3'] = '.'
loadings_table.loc[np.abs(loadings_table['fact. 4']) < pca_cut, 'fact. 4'] = '.'
# iwave
loadings_table.loc[np.abs(loadings_table['iwave fact. 1']) < iwave_cut, 'iwave fact. 1'] = '.'
loadings_table.loc[np.abs(loadings_table['iwave fact. 2']) < iwave_cut, 'iwave fact. 2'] = '.'
loadings_table.loc[np.abs(loadings_table['iwave fact. 3']) < iwave_cut, 'iwave fact. 3'] = '.'
loadings_table.loc[np.abs(loadings_table['iwave fact. 4']) < iwave_cut, 'iwave fact. 4'] = '.'
# niwave
loadings_table.loc[np.abs(loadings_table['niwave fact. 1']) < niwave_cut, 'niwave fact. 1'] = '.'
loadings_table.loc[np.abs(loadings_table['niwave fact. 2']) < niwave_cut, 'niwave fact. 2'] = '.'
loadings_table.loc[np.abs(loadings_table['niwave fact. 3']) < niwave_cut, 'niwave fact. 3'] = '.'
loadings_table.loc[np.abs(loadings_table['niwave fact. 4']) < niwave_cut, 'niwave fact. 4'] = '.'


print(round(loadings_table,2).to_latex(escape=True, index=True, index_names=False, column_format='lrrrrrrrrrrrr', ))